#### Instala os módulos necessários

In [ ]:
%pip install numpy
%pip install pandas
%pip install scikit-learn

#### Importa os módulos necessários

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV

#### Análise exploratória, feature engineering, treinamento e avaliação do modelo

In [2]:
raw = pd.read_csv(r"_datasets/dataset.csv")

In [ ]:
raw.head(5)

In [ ]:
raw.info()

A variável "total_bedrooms" apresenta 207 nulos, dessa forma, realizaremos a inputação dos dados utilizando a função [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) do SKLearn para preencher os valores ausentes. De modo geral, a função aplica a média (ou outra estatistica de sua preferência) dos "k" vizinhos mais próximos do dado faltante e atribui o resultado ele.

Entretanto, para isso, precisaremos converter a variável "ocean_proximity" para valores numéricos, visto que o KNNImputer não consegue lidar com textos. Para isso, assumiremos que quanto mais próximo o imóvel está em relação ao oceano, maior será o seu valor. Sendo assim, precisaremos descobrir quais são as categorias e a média de valor dos imóveis para cada uma delas.

In [ ]:
raw.groupby(by="ocean_proximity")["median_house_value"].mean().sort_values(ascending=False)

Note que a variável "ocean_proximity" possui 5 categorias: ISLAND, ou ilha; NEAR BAY, ou próximo ao baia; NEAR OCEAN, ou próximo ao oceano; <1H OCEAN, ou menos de uma hora de distância do oceano; e, INLAND, ou interior. Neste caso, atribuiremos pesos (1.0, 0.8, 0.6, 0.4, 0.2) a cada categoria, priorizando aquelas em que o imóvel está mais próximo do oceano, visto que a nossa hipótese é que imóveis localizados mais próximos do oceano em média são mais valorizados.

In [6]:
raw["ocean_proximity"] = raw["ocean_proximity"].apply(lambda x: 1.0 if x == "ISLAND" else x)
raw["ocean_proximity"] = raw["ocean_proximity"].apply(lambda x: 0.8 if x == "NEAR BAY" else x)
raw["ocean_proximity"] = raw["ocean_proximity"].apply(lambda x: 0.6 if x == "NEAR OCEAN" else x)
raw["ocean_proximity"] = raw["ocean_proximity"].apply(lambda x: 0.4 if x == "<1H OCEAN" else x)
raw["ocean_proximity"] = raw["ocean_proximity"].apply(lambda x: 0.2 if x == "INLAND" else x)

In [ ]:
raw.groupby(by="ocean_proximity")["median_house_value"].mean().sort_values(ascending=False)

Neste momento, o conjunto de dados possui apenas dados numéricos, ou seja, podemos aplicar a função KNNImputer para preencher os dados faltantes. Não somente, aproveitaremos a oportunidade para utilizar a função [GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) do SKLearn, que de modo geral, a partir da definição de uma lista de hiperparâmetros, realiza a busca pela melhor combinação para treinar o modelo.

In [ ]:
k_values = [2, 3, 4, 5]

custom_parameters = {
    "n_estimators" : [10, 25, 50, 100],
    "max_depth" : [None, 1, 2, 3, 4, 5],
    "min_samples_split" : [None, 1, 2, 3, 4, 5],
    "min_samples_leaf" : [None, 1, 2, 3, 4, 5]
}

for k in k_values:
    stage = raw 

    cols_names = list(stage.columns)

    imputer = KNNImputer(n_neighbors=k)

    stage = pd.DataFrame(imputer.fit_transform(stage), columns=cols_names)

    x = stage[["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "ocean_proximity"]]

    y = stage["median_house_value"]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

    random_forest = RandomForestRegressor(random_state=42)

    grid_search = GridSearchCV(estimator=random_forest, param_grid=custom_parameters, cv=5, scoring="neg_mean_squared_error", verbose=1, n_jobs=6)

    grid_search.fit(x_train, y_train)

    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(x_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"################################################## KNNImputer = {k} ##################################################")
    print(f"Erro Absoluto Médio (MAE): {mae}")
    print(f"Erro Quadrático Médio (MSE): {mse}")
    print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")
    print(f"Coeficiênte de Determinação (R²): {r2}")
    print(f"Melhores Hiperparâmetros: {grid_search.best_params_}")
    print(f"######################################################################################################################\n\n")